In [1]:
import torch
import torch.nn.functional as F
import sys
import numpy as np

sys.path.append('/home/hxcai/cell_type_specific_CRE')
from MPRA_exp.datasets import SeqLabelDataset
from MPRA_exp.utils import *
from sei_framework.model.sei import Sei

/home/hxcai/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'sei_framework'

In [2]:
def get_embedding(model, test_data_loader, device='cuda'):
    model = model.to(device)
    y_true = []
    y_pred = []
    embedding = []
    with torch.no_grad():
        model.eval()
        for (x, y) in tqdm(test_data_loader):
            x = x.to(device)
            x_rc = onehots_reverse_complement(x).to(device)
            
            pred_1, emb_1 = model.get_embedding(x)
            pred_2, emb_2 = model.get_embedding(x_rc)
            pred = (pred_1 + pred_2) / 2
            emb  = (emb_1 + emb_2) / 2
            # print(emb.cpu().detach().numpy())

            y_true.extend(y.cpu().detach().numpy())
            y_pred.extend(pred.cpu().detach().numpy())
            embedding.extend(emb.cpu().detach().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    embedding = np.array(embedding)
    return y_true, y_pred, embedding

In [3]:
trained_model_path = '../sei_framework/model/sei.pth'
result_dir = './data'

sei_model = Sei().eval()
state_dict = torch.load(trained_model_path) 
new_state_dict = {k.replace('module.model.', ''): v for k, v in state_dict.items()}
sei_model.load_state_dict(new_state_dict)

dataset = SeqLabelDataset(table_dir='../data/lentiMPRA/joint_library_table_s.csv', input_column='seq', output_column=None, seq_pad_len=4096)
test_data_loader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=False)
y_true, y_pred, embedding = get_embedding(sei_model, test_data_loader)

100%|██████████| 223/223 [03:14<00:00,  1.15it/s]


In [4]:
np.save(f'{result_dir}/sei_joint_pred.npy', y_pred)
np.save(f'{result_dir}/sei_joint_embedding.npy', embedding)

In [5]:
np.where(embedding[0] - embedding[1] != 0)

(array([    4,     5,     6, ..., 15353, 15354, 15355]),)